# DCIR Setup Leveraging EDB Configuration Format

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import os
import tempfile

from ansys.aedt.core import Hfss3dLayout
from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example BGA Package design.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source=r"edb/BGA_Package.aedb", local_path=temp_folder.name)

## Load example layout

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 04:05:28.561431


INFO:Edb:Star initializing Edb 04:05:28.561431


PyEDB INFO: Edb version 2025.2


INFO:Edb:Edb version 2025.2


...

INFO:Edb:Builder was initialized.


PyEDB INFO: open_edb completed in 8.7285 seconds.


INFO:Edb:open_edb completed in 8.7285 seconds.


PyEDB INFO: EDB initialization completed in 8.8085 seconds.


INFO:Edb:EDB initialization completed in 8.8085 seconds.


## Create config file

Define Component with solderballs.

In [5]:
cfg_components = [
    {
        "reference_designator": "FCHIP",
        "part_type": "other",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.1mm",
            "height": "0.085mm",
            "chip_orientation": "chip_up",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
    {
        "reference_designator": "BGA",
        "part_type": "io",
        "solder_ball_properties": {
            "shape": "cylinder",
            "diameter": "0.45mm",
            "height": "0.45mm",
            "chip_orientation": "chip_down",
        },
        "port_properties": {
            "reference_offset": 0,
            "reference_size_auto": False,
            "reference_size_x": 0,
            "reference_size_y": 0,
        },
    },
]

Define Pin Groups on Components.

In [6]:
cfg_pin_groups = [
    {"name": "BGA_VSS", "reference_designator": "BGA", "net": "VSS"},
    {"name": "BGA_VDD", "reference_designator": "BGA", "net": "VDD"},
]

Define sources.

In [7]:
cfg_sources = [
    {
        "name": "FCHIP_Current",
        "reference_designator": "FCHIP",
        "type": "current",
        "magnitude": 2,
        "distributed": True,
        "positive_terminal": {"net": "VDD"},
        "negative_terminal": {"nearest_pin": {"reference_net": "VSS", "search_radius": 5e-3}},
    },
    {
        "name": "BGA_Voltage",
        "reference_designator": "BGA",
        "type": "voltage",
        "magnitude": 1,
        "positive_terminal": {"pin_group": "BGA_VDD"},
        "negative_terminal": {"pin_group": "BGA_VSS"},
    },
]

Define DCIR setup.

In [8]:
cfg_setups = [
    {
        "name": "siwave_dc",
        "type": "siwave_dc",
        "dc_slider_position": 1,
        "dc_ir_settings": {"export_dc_thermal_data": True},
    }
]

Create final configuration.

In [9]:
cfg = {
    "components": cfg_components,
    "sources": cfg_sources,
    "pin_groups": cfg_pin_groups,
    "setups": cfg_setups,
}

Create the config file.

In [10]:
file_json = os.path.join(temp_folder.name, "edb_configuration.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

## Apply Config file

Apply configuration to the example layout

In [11]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating boundaries finished. Time lapse 0:00:00.006599


INFO:Edb:Updating boundaries finished. Time lapse 0:00:00.006599


PyEDB INFO: Updating nets finished. Time lapse 0:00:00


INFO:Edb:Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00.015845


...

PyEDB INFO: Placing probes finished. Time lapse 0:00:00


INFO:Edb:Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


INFO:Edb:Applying operations completed in 0.0000 seconds.


True

Save and close EDB.

In [12]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.0160 seconds.


INFO:Edb:Save Edb file completed in 0.0160 seconds.


PyEDB INFO: Close Edb file completed in 0.0158 seconds.


INFO:Edb:Close Edb file completed in 0.0158 seconds.


True

The configured EDB file is saved in a temp folder.

In [13]:
print(temp_folder.name)

C:\Users\ansys\AppData\Local\Temp\tmpee87qt8n.ansys


## Load edb into HFSS 3D Layout.

In [14]:
h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


INFO:Global:Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.22.dev0.


INFO:Global:PyAEDT version 0.22.dev0.


PyAEDT INFO: Initializing new Desktop session.


...

INFO:Global:Active Design set to 0;BGA_Package


PyAEDT INFO: Active Design set to 0;BGA_Package


INFO:Global:Active Design set to 0;BGA_Package


PyAEDT INFO: Aedt Objects correctly read


INFO:Global:Aedt Objects correctly read


Solve.

In [15]:
h3d.analyze(setup="siwave_dc")

PyAEDT INFO: Project BGA_Package Saved correctly


INFO:Global:Project BGA_Package Saved correctly


PyAEDT INFO: Solving design setup siwave_dc


INFO:Global:Solving design setup siwave_dc


PyAEDT INFO: Design setup siwave_dc solved correctly in 0.0h 0.0m 30.0s


INFO:Global:Design setup siwave_dc solved correctly in 0.0h 0.0m 30.0s


True

Plot insertion loss.

In [16]:
h3d.post.create_fieldplot_layers_nets(layers_nets=[["VDD_C1", "VDD"]], quantity="Voltage", setup="siwave_dc")

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpee87qt8n.ansys\edb\BGA_Package.aedt.


INFO:Global:Parsing C:\Users\ansys\AppData\Local\Temp\tmpee87qt8n.ansys\edb\BGA_Package.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpee87qt8n.ansys\edb\BGA_Package.aedt correctly loaded. Elapsed time: 0m 0sec


INFO:Global:File C:\Users\ansys\AppData\Local\Temp\tmpee87qt8n.ansys\edb\BGA_Package.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.08872365951538086


...

INFO:Edb:open_edb completed in 0.0797 seconds.


PyEDB INFO: EDB initialization completed in 0.0797 seconds.


INFO:Edb:EDB initialization completed in 0.0797 seconds.


PyAEDT INFO: Active Design set to 0;BGA_Package


INFO:Global:Active Design set to 0;BGA_Package


Shut Down Electronics Desktop

In [17]:
h3d.close_desktop()

PyAEDT INFO: Desktop has been released and closed.


INFO:Global:Desktop has been released and closed.


True

All project files are saved in the folder ``temp_file.dir``. If you've run this example as a Jupyter notebook you
can retrieve those project files.